In [5]:
# connect with google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# access data 
%cd /content/drive/MyDrive/Daten/

/content/drive/MyDrive/Daten


In [ ]:
# check number of files 
import os 

num_files = len(os.listdir("."))
print("Number of files in the folder: ", num_files)

Number of files in the folder:  253


In [ ]:
# check if any folder is empty
empty_folders = []

for root, dirs, files in os.walk('.'):
    if not dirs and not files:
        empty_folders.append(root)

print("Empty folders:")
for folder in empty_folders:
    print(folder)


Empty folders:


In [ ]:
import pandas as pd

dataset = pd.DataFrame(columns=['Filename', 'Type', 'Content'])

for entry in sorted(os.listdir('.')):
    entry_path = os.path.join('.', entry)
    if os.path.isdir(entry_path):
        # Entry is a folder, add XML files in the folder to the dataset
        for file in sorted(os.listdir(entry_path)):
            if file.endswith('.xml'):
                file_path = os.path.join(entry_path, file)
                with open(file_path, 'r') as f:
                    content = f.read()
                dataset = dataset.append({'Filename': file, 'Type': 'XML', 'Content': content}, ignore_index=True)
    elif os.path.isfile(entry_path) and entry.endswith('.xml'):
        # Entry is an XML file, add it to the dataset
        with open(entry_path, 'r') as f:
            content = f.read()
        dataset = dataset.append({'Filename': entry, 'Type': 'XML', 'Content': content}, ignore_index=True)

print(dataset.head())


things to do: 
- connect to github 
- create and store dataset 
- do some descriptive analysis: number of articles per newspaper, number of newspapers, number of topics per newspaper, etc. 
- do filtering: german newspapers only, DA related topics only 
- topic analysis: run the filtered dataset through a generic topic model 